# One-vs-All Classification

## Table of Contents
- [Introduction](#intro)
- [The Dataset](#dataset)
- [The Cost Function and Gradient](#cost_and_gradient)

<a id='intro'></a>
### Introduction
In a [previous article](https://github.com/marty-vanhoof/Maching_Learning/blob/master/Logistic_Regression/Logistic_Regression.ipynb), we talked in detail about logistic regression in the case of binary classification; that is, given a set of training data $\{ \, ( \mathbf{x}_1,y_1 ), ( \mathbf{x}_2,y_2 ) \ldots,  ( \mathbf{x}_m,y_m ) \, \}$, logistic regression finds the parameters $\theta_0, \theta_1, \ldots, \theta_n$ of a function $h_\theta(\mathbf{x})$ that estimates the probability of a class label $y=1$ given a feature vector $\mathbf{x}$, and so the function $1 - h_\theta(\mathbf{x})$ estimates the probability of a class label $y=0$ given $\mathbf{x}$.  We can then predict binary outcomes by specifying a probability threshold, and the threshold we use is given as follows

$$ \mathrm{prediction} =
\begin{cases}
1 & \mathrm{if} \quad P(y = 1 \,|\, \mathbf{x} ; \theta) \geq 0.5 \\ 
0 & \mathrm{if} \quad P(y = 0 \,|\, \mathbf{x} ; \theta) < 0.5
\end{cases} 
$$

One-vs-all classification extends logistic regression to the case of classifying more than two class labels by considering a sequence of binary classifications.  Suppose the target variable $y$ can be labeled in $k$ possible ways, so $y \in \{1, 2, \ldots, k \}$ with $k \geq 2$.  We can turn this into a sequence of binary classification problems and use logistic regression at each step.  

First separate the labels $\{1, 2, \dots, k \}$ into either 1 or $\{2, \dots, k \}$ and fit a standard logistic regression model to this.  This logistic regression model will find a function $h_\theta^{(1)}(\mathbf{x})$ that classifies the data into either 1 or $\{ 2, \ldots, k \}$.

Next, separate the labels into either 2 or $\{ 1, 3, \dots, k \}$ and fit another logistic regression model to this, resulting in a function $h_\theta^{(2)}(\mathbf{x})$ that classifies the data into either 2 or $\{ 1, 3, \dots, k \}$.
And so on...the last step will fit another logistic regression model that finds a function $h_\theta^{(k)}(\mathbf{x})$ that classifies the data into either $k$ or $\{ 1, 2, \ldots, k-1 \}$.

The result is that we have $k$ classifiers $h_\theta^{(1)}(\mathbf{x}), h_\theta^{(2)}(\mathbf{x}), \ldots, h_\theta^{(k)}(\mathbf{x})$ each trying to estimate the probability of a class label $y \in \{ 1, 2, \ldots, k \}$ given $\mathbf{x}$.  That is,

$$
h_\theta^{(i)}(\mathbf{x}) = P(y = i \,|\, \mathbf{x}; \theta) \,, \quad i = 1,2, \ldots, k
$$

Then to make a prediction on a feature vector $\mathbf{x}$, pick the class $i$ that gives maximum probability

$$
\max \limits_i h_\theta^{(i)}(\mathbf{x}) 
$$

A nice discussion of one-vs-all classification can be found in [this video](https://www.youtube.com/watch?v=BzSsQWhDRXE&list=PLZ9qNFMHZ-A4rycgrgOYma6zxF4BZGGPW).

<a id='dataset'></a>
### The Dataset
As usual, we will be working through an example from one of the programming exercises in Andrew Ng's machine learning course.  We will implement one-vs-all logistic regression to recognize hand-written digits.  The [dataset](https://github.com/marty-vanhoof/Maching_Learning/blob/master/data/logReg_data3.mat) contains 5000 training examples of handwritten digits, which is a subset of the MNIST database of handwritten digits available [here](http://yann.lecun.com/exdb/mnist/).  This picture below is an example of some of the digits.
<br/>
<img src="digits.png" height="400" width="400">
<br/>
Each training example (each little box) is a 20 pixel by 20 pixel grayscale image of the digit.  The image can be represented as a 400 dimensional vector $\mathbf{x}_i$ of floating-point numbers, where each number represents the grayscale intensity of a pixel at that location.  Each of these training examples becomes a single row in a feature matrix $X$.  The class labels are in the target vector $\mathbf{y} = (y_1, y_2, \ldots, y_{400})^T$, where each $y_i$ can be one of the numbers 0 through 9

$$
X =
\begin{bmatrix}
\, - \,\, \mathbf{x}_1^T - \, \\
\, - \,\, \mathbf{x}_2^T - \, \\
\vdots \\
\, - \,\, \mathbf{x}_{400}^T - \,
\end{bmatrix} \, , \quad
\mathbf{y} =
\begin{bmatrix}
y_1 \\ y_2 \\ \vdots \\ y_{400} 
\end{bmatrix} 
$$ 

The dataset is in a MATLAB format, so we have to use a function `loadmat()` in the SciPy library in order for Pandas to interpret it.

In [1]:
import numpy as np  
import pandas as pd 
import os
import matplotlib.pyplot as plt  
from scipy.io import loadmat  
%matplotlib inline

filepath = os.getcwd() + '/logReg_data3.mat'
data = loadmat('logReg_data3.mat')
data

{'X': array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]),
 '__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 'y': array([[10],
        [10],
        [10],
        ..., 
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

.

We should make sure that $X$ and $\mathbf{y}$ have the proper shapes.

In [2]:
print(data['X'].shape, data['y'].shape)

(5000, 400) (5000, 1)


<a id='cost_and_gradient'></a>
### The Cost Function and Gradient

Since we are going to use one-vs-all logistic regression to classify the digits 0 through 9, we will need to train 10 separate logistic regression models.  We will re-use the logistic cost function and the gradient code from before, and we will also implement the regularized versions of these functions.  Recall that the regularized cost function is given as

$$ J(\theta) = \frac{1}{m} \sum_{i=1}^m [- y_i \ln h_\theta(\mathbf{x}_i) - (1 - y_i) \ln (1 - h_\theta(\mathbf{x}_i))] + \frac{\lambda}{2m} \sum_{j=1}^n \theta_j^2 \,, $$

and the Python function below implements it.  Note that `regularized_logReg_cost()` also imports the standard (unregularized) logistic cost function and uses that instead of repeating the code from before.

In [7]:
def regularized_logReg_cost(theta, X, y, lambda_):
    '''Compute the regularized cost funtion.  This is almost the same as
    logReg_cost() from before, but now we're adding a regularization term.'''
    
    # import the logistic regression cost function that we wrote before
    from log_reg_scripts import logReg_cost
    
    unreg_cost = logReg_cost(theta, X, y)
    # convert theta back to a numpy array since logReg_cost converts it to a numpy matrix
    theta = np.array(theta)
    reg_term = lambda_ / (2*len(y)) * sum(theta[1:]**2) 
    return unreg_cost + reg_term